In [0]:
%pip install -U mlflow langchain langgraph-checkpoint langchain_core langgraph pydantic databricks-langchain
# dbutils.library.restartPython()

In [0]:
import mlflow
from databricks_langchain import ChatDatabricks

mlflow.langchain.autolog()

In [0]:
from typing import Iterator, Dict, Any
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    ToolMessage,
    MessageLikeRepresentation,
)
from mlflow.types.llm import ChatCompletionRequest, ChatCompletionResponse, ChatChoice, ChatMessage
from random import randint
from dataclasses import asdict
import logging

import json

def create_flexible_chat_completion_response(content: str, id: int = 0) -> Dict:
    return asdict(ChatCompletionResponse(
        choices=[ChatChoice(message=ChatMessage(role="assistant", content=content))],
        custom_outputs={
            "id": id
        },
    ))

def wrap_output(stream: Iterator[MessageLikeRepresentation]) -> Iterator[Dict]:
    """
    Process and yield formatted outputs from the message stream.
    The invoke and stream langchain functions produce different output formats.
    This function handles both cases.
    """
    for event in stream:
        # the agent was called with invoke()
        if "messages" in event:
            output_content = ""
            for msg in event["messages"]:
                output_content += msg.content
            # Note: you can pass additional fields from your LangGraph nodes to the output here
            yield create_flexible_chat_completion_response(content=output_content, id=randint(100000000, 10000000000))
        # the agent was called with stream()
        else:
            for node in event:
                for key, messages in event[node].items():
                    if isinstance(messages, list):
                        for msg in messages:
                            # Note: you can pass additional fields from your LangGraph nodes to the output here
                            yield create_flexible_chat_completion_response(content=msg.content, id=randint(100000000, 10000000000))
                    else:
                        logging.warning(f"Unexpected value {messages} for key {key}. Expected a list of `MessageLikeRepresentation`'s")
                        yield create_flexible_chat_completion_response(content=str(messages))

In [0]:
func_name = [
    "trainer_catalog.03_data_analysis_by_gen_ai_for_nssol.product_with_many_inquiries",
    "trainer_catalog.05_vector_search_index_for_nssol.product_docs_vector_search"
             ]

In [0]:
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

In [0]:
# Databricks Function クライアントの作成
client = DatabricksFunctionClient()

# UCFunctionToolkit を利用して、Unity Catalog 関数をツールとして登録
toolkit = UCFunctionToolkit(
    function_names=func_name,
    client=client
)

tools = toolkit.tools



In [0]:
from typing import Annotated

from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm = ChatDatabricks(endpoint="openhack-gpt-4o")
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [0]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

In [0]:
from langchain_core.runnables import RunnableGenerator
from langgraph.prebuilt import create_react_agent

# Create the agent with the system message if it exists
agent_with_raw_output = create_react_agent(llm_with_tools, tools)

agent = agent_with_raw_output| RunnableGenerator(wrap_output)

In [0]:
mlflow.models.set_model(agent)